本文内容主要来自:
- https://arxiv.org/pdf/1607.00578.pdf

## paper简介
我们发现，在机器翻译中使用连续词向量表达符号存在一个潜在的弱点:<br>
- The continuous vector representation, or a word embedding vector, of a symbol encodes multiple dimensions of similarity, equivalent to encoding more than one meaning of the word. 也就是说，我们使用一个连续词向量来描述一个单词的多个意思（一个单词和其他单词在不同方面的相识度）。这导致，在机器翻译的过程中，他将根据原文句子提供的上下文花费大量的精力来处理原文和目标文单词之间的歧义。<br>

基于这个发现，我们在这里提出:
- contextualize the word embedding vectors using a nonlinear bag-of-words representation of the source sentence.
- to represent special tokens(such as numbers, proper nouns and acronyms) with typed symbols to facilitate translating those words that are not well-suited to be translated via continuous vectors.

## 方法简介
一般的情况下，神经机器翻译的第一个步骤就是把原始的符号转换成对应的高维连续向量（词向量）。而在这个转换过程中，每个单词是独立完成转换的，也就是说每个单词并不受所在句子的上下文影响。


## Contextualized Word Embedding Vectors
##### Multiple Dimensions of Similarity
高维词向量的一个重要的特征就是: it encodes multiple dimensions of similarities. 也就是说它对多维相似度进行编码（也就是说，他同事对一个单词的多个意思进行编码）。这有利于神经元网络处理一词多义的问题。上面这种现象，我们可以通过下面这个方法观察到：<br>
给定高维词向量x'，我们找到离这个词向量最近的N-1个词向量$x^1,...,x^{N-1}$。使用PCA进行降维，这是我们可以观察不同方向下的最近邻。可以发现其结果是不同的，从而证实上面的猜想。
![multi dimension similarity](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/multi_dims_similarity.png)
##### Contextualized Word Embeddings
multiple dimensions of Similarity的问题意味着，神经元网络需要根据单词所在的上下文信息，来消除这个该单词词向量的二义性。在机器翻译中，原文和目标文的单词是分开处理的。在encoder中，一个单词，可以通过所在句子的其他单词，来消除他的二义性。而对于decoder来说，他可以通过上一个单词，或者是原文的内容来做这个处理。<br>
正如Weaver在1949年所说的那样，一个单词的二义性，大部分可以通过其周围的单词来消除。对于机器翻译的问题，这意味着，不管是encoder还是decoder，对于每个单词，他需要记住其前面出现的单词，从而来消除自身的二义性。但这并不是我们想要的。我们希望他能够集中精力与它的翻译工作？？<br>
为了解决这个问题，我们建议在把词向量扔进RNN之前，进行如下图中的处理：
![contextualization](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/contextualization.png)
这个方法的中心思想是，根据上下文，通过掩盖掉词向量的一些维度来实现消除二义性的效果。具体过程如下：
- 我们用$c^x$来代表原文句子的上下文，他由句子中各个单词通过非线性转换后的求平均而得：$c^x=\frac{1}{T}\sum^T_{t=1}NN_\theta(x_t)$其中NN为一个前向非线性神经网络。
- 随后，我们根据这个$c^x$，为每个词向量计算一个mask。并在这个词向量送入rnn之前，对其进行处理：$x_t<-x_t\bigodot \sigma(W_xc^x+b_x)$,$y_t<-y_t\bigodot \sigma(W_yc^x+b_y)$。其中$\sigma$用于生成mask。

## Symbolization
连续高维向量带来的另一个问题是对特定名词，数值和稀有词的处理<br>
稀有词因为在训练过程中出现的次数，从而导致其词向量不能很好的被探着（也就是说起词向量的代表性不强）。通常在实践中，我们会使用unknown来代表所有的稀有词。这种方法在语言模型中效果不错，但是在翻译模型中确是不是很美。<br>
而对于数字，我们知道数字的意义是不受其周围环境的影响的。所以使用通过我们的方法求得的数字的词向量，可能带入很多不必要的噪音。<br>
专有名词，同时包含了上面提到的两个问题。除了少数非常常用的专有名词之外，大多数专有名词都是比较稀有的。同时在对专有名词进行翻译的时候会带入多于的噪音。<br>
基于上面这三个问题及其内在的原因，我们建议，把上面三类词汇，区别于其他单词进行处理。
##### Previous Approaches
大多数时候，原文中的特殊词汇会通过查表的方式被替换为对应的目标文<br>
基于这种想法Jean et al. Jean et al. (2015a)提出了一系列的启发式方法在基于Attention机制的翻译器中处理这些特殊词汇。在这些方法中，使用Attention机制中的权值来确定生成的特殊字符所对应的原文字符（比如取最大权值对应的字符）。然后通过查表，来确定目标特殊字符。<br>
除此之外，Luong et al. Luong et al（2015b）也提出了另外的一种处理方法，且这种方法不依赖于Attention机制。他使用二外的alignment(对齐)机制，来确定训练集中，原文和目标文中特殊单词的对齐关系。这个对齐机制为原文的特殊字符分配有编号的标识符。比如，当原文中出现第一个特殊字符的时候，对齐机制找到其于目标文中对应的单词，并把他标记为$<UNK1>$(这里并不关心该词在目标文中所处的位置）。在训练的时候，我们使用对齐机制中的这些特殊标识来取代训练集中出现特殊字符。而在测试的时候，给定原文和生成的目标文，其中出现的未知符，会根据对应原文单词查表得到。
###### Symbolization of Proper Nouns, Digits and Rare Words
我们这里提出的方法，是对上面提到的Luong et al. Luong et al（2015b）方法的一个扩展。上面的方法中，我们只使用一个特殊标识符，在这里我们使用三个，他们分别是：
- $⟨N⟩_n$: Digit
- $⟨S⟩_n$: Proper noun
- $⟨C⟩_n$: Acronym

其中，下标指代单词在原文中出现的顺序。
![symbolization](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/symbolization.png)

## 存在问题的句子
1-5 We conjecture that only a few axes in this high-dimensional space are relevant given a source sentence and that we can remove much of the ambiguity in the choice of words by restricting, or turning off, most of the irrelevant dimensions. <br>
3-2 we can qualitatively check this phenomenon of multiple dimensions of similarities by inspecting a local chart of the manifold on which the word embedding vectors reside<br>
3-3 In other words, the encoder and decoder must sacrifice their capacity in disambiguating the words. This is undesirable, as what we truly want the encoder and decoder to do is to capture the higher level compositional structures of a sentence that are necessary for translation.??<br>
4-1 The use of continuous vectors as an intermediate representation of source and target sentences in neural machine translation greatly improves generalization of machine translation by avoiding the issue of data sparsity --Sec. 5.2.1 in Cho (2015)<br>
4-2 15年提出的处理特殊字符的一种方法。arXiv preprint arXiv:1410.8206.<br>